# Gen AI - RAG Notebook

## 🎯 Objectives

The objective of this notebook is to do Retrieval-Augmented Generation thanks to langchain framework

You will learn how to use rag using langgraph

## ⚙️ Setup

- Go to your terminal 
- Run the script `00-init.sh`
- Inside this notebook select the good kernel
- Be sure to have your Cohere API Key as environment variable

In [1]:
!pip install langchain
!pip install -qU langchain-core
!pip install -U langchain-cohere
!pip install -qU langgraph
!pip install -qU langchain_community beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

COHERE_API_KEY = os.environ.get('COHERE_API_KEY')

## RAG

Your goal is to do RAG on the content of this blog: https://lilianweng.github.io/posts/2023-06-23-agent/

To do that, you will need to:

- Use WebBaseLoader of langchain to load the content
- Split the document to have 1000 characters (use RecursiveCharacterTextSplitter)
- Embed the contents thanks to a Cohere embedding model
- Store the vectors in a in-memory vectorstore
- Init a langraph instance
- Ask your question
- Print the sources

#### Load the content

In [3]:
from langchain_community.document_loaders import WebBaseLoader

# URL du blog à charger
blog_url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

# Créer une instance de WebBaseLoader
loader = WebBaseLoader(blog_url)

# Charger le contenu du blog
docs = loader.load()

# Extraire le texte du document (exemple générique, nécessite un parsing plus spécifique)
text_content = docs[0].page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# Load contents of the blog
"""Create a langchain WebBaseLoader and load docs"""

print("Number of document: ", len(docs)) # Should be 1
display(docs) # Should be a list of Document objects

Number of document:  1


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

#### Split the document

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialiser le splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Taille des morceaux
    chunk_overlap=100  # Chevauchement pour garder du contexte
)

# Diviser le texte en segments
all_splits = text_splitter.split_text(text_content)


In [6]:
"""
    Split the document into smaller chunks.
    Use RecursiveCharacterTextSplitter.
    The output should be a list of Document objects.
"""

print("Number of document: ", len(all_splits)) # Should be more than 30
display(all_splits) # Should be a list of Document objects

Number of document:  67


["LLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences",
 'Generative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences\n\n\n\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-

#### Embed and store the contents

Create the embeddings

In [7]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")

Create the in memory vectorstore

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Add chunks in the vectorstore

In [10]:
from langchain_core.documents import Document

documents = [Document(page_content=chunk) for chunk in all_splits]

_ = vector_store.add_documents(documents=documents) # Add the documents to the vector store

#### Get info from the docs 

Create a graph

In [13]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")
from langchain.chat_models import init_chat_model

llm = init_chat_model("command-r-plus", model_provider="cohere")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    """
        Retrieve relevant documents from the vector store.
        Do a similarity search on the question in the state and retrieve the documents.
    """
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/Users/julienrameaux/Desktop/Dauphine_2425_Groupe_1/venv/lib/python3.12/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#### Ask a question

In [14]:
"""Ask a relevant question and get the answer"""
question = "What are the differents task decompositions in a planning ?"  # Remplace par ta question
response = graph.invoke({"question": question})

# Afficher la réponse
print(response["answer"])

Task decomposition can be achieved through Chain of Thought (CoT) or Tree of Thoughts (ToT) methods, using LLM prompting, task-specific instructions, or human input. CoT breaks tasks into smaller steps, while ToT explores multiple reasoning paths, creating a tree structure.


Display sources

In [15]:
"""Display the sources used to get the answer"""
sources = response["context"]  # Les documents récupérés
print("\nSources utilisées :")
for i, doc in enumerate(sources):
    print(f"Source {i+1}: {doc.page_content[:500]}...")  # Afficher un extrait


Sources utilisées :
Source 1: Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big ta...
Source 2: Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting

## 🧠 Go beyond

### 1. The system prompt

#### 1.1 When is it used ?

Your **system prompt** will be provided to the LLM along with the selected relevant documents and the conversation history **each time** the user will ask a question. 

It should include context about your documents, and guidelines about how to answer the user. Here is an example : 

« You are a Cybersecurity Information Assistant, you are able to answer questions related to .. ​
A L'Oréal employee has a question for you. To formulate your answer, follow these rules:  ​
1) Answer him in the language he speaks to you.    ​
2) Use only the context below to answer his question. ​
3) If the context I've given you doesn't allow you to answer, answer that you don't know.   »     

#### 1.2 Edit System Prompt

**It's your time to play**

In [16]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate
# Define prompt for question-answering
system_prompt = """You are an AI assistant specialized in Autonomous AI Agents.
You will answer questions based on the retrieved context.

Follow these rules:
1) Respond in the language used by the user.
2) Use only the provided context to formulate your answer.
3) If the context is insufficient, simply state that you do not know.
4) Be clear and concise in your explanations.

Here is the user question and retrieved context:
Question: {question}
Context: {context}
"""

# Création du prompt
prompt = PromptTemplate.from_template(system_prompt)

from langchain.chat_models import init_chat_model

llm = init_chat_model("command-r-plus", model_provider="cohere")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    """
        Retrieve relevant documents from the vector store.
        Do a similarity search on the question in the state and retrieve the documents.
    """
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

### 2. The filtering prompt

An optional filtering step is available by adding a *filtering_prompt*. 
The filtering prompt decides to keep a document as relevant or not, by adding a boolean value True/False to decide wether or not to keep it.

#### 2.1 Example of a filtering_prompt

« Your role is to rate the helpfulness of a document given a user message. The rating you can give is either True or False. 
Here's how to approach the task:

**Direct definition:** If the document explicitely defines the term or concept asked about in the user message, return 'True'.

**Mention Without Definition:** If the document mentions the term but does not provide a definition or explanation, return 'False'.

**Irrelevant Content:** If the document is unrelated to the user's query, or if the user's message is a greeting or conversational phrase (e.g. 'Hello'), return 'False'.

**When in doubt, Default to False:** If unsure, return 'False' by default. »

#### 2.2 Create a Filtering Prompt

**Create a filtering prompt to get only relevant info**

In [20]:
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Initialisation du modèle LLM
llm = init_chat_model("command-r-plus", model_provider="cohere")

#  Prompt pour filtrer les documents
filtering_prompt_template = """Your role is to rate the helpfulness of a document given a user message.
The rating you can give is either True or False.

**Direct definition:** If the document explicitly defines the term or concept asked about in the user message, return 'True'.

**Mention Without Definition:** If the document mentions the term but does not provide a definition or explanation, return 'False'.

**Irrelevant Content:** If the document is unrelated to the user's query, or if the user's message is a greeting or conversational phrase (e.g. 'Hello'), return 'False'.

**When in doubt, Default to False:** If unsure, return 'False' by default.

---

**User Message:** {question}
**Document:** {document}

Respond with only 'True' or 'False'.
"""
filtering_prompt = PromptTemplate.from_template(filtering_prompt_template)

# Prompt principal pour générer la réponse
system_prompt = """You are an AI assistant specialized in Autonomous AI Agents.
You will answer questions based on the retrieved context.

Follow these rules:
1) Respond in the language used by the user.
2) Use only the provided context to formulate your answer.
3) If the context is insufficient, simply state that you do not know.
4) Be clear and concise in your explanations.

Here is the user question and retrieved context:
Question: {question}
Context: {context}
"""
qa_prompt = PromptTemplate.from_template(system_prompt)

# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    """Récupérer les documents pertinents depuis le vector store."""
    retrieved_docs = vector_store.similarity_search(state["question"])  # Récupération brute
    return {"context": retrieved_docs}


def filter_documents(state: State):
    """Filtrer les documents selon leur pertinence en utilisant le filtering prompt."""
    relevant_docs = []
    for doc in state["context"]:
        message = filtering_prompt.format(question=state["question"], document=doc.page_content)
        response = llm.invoke(message).content.strip()

        if response.lower() == "true":
            relevant_docs.append(doc)

    return {"context": relevant_docs}

def generate(state: State):
    """Générer une réponse en utilisant les documents filtrés."""
    if not state["context"]:  # Aucun document pertinent
        return {"answer": "Je ne sais pas, le contexte est insuffisant."}

    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = qa_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, filter_documents, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


question = "Quels sont les différents types de décompositions de tâches en planification ?"
response = graph.invoke({"question": question})

# Affichage de la réponse
print(response["answer"])


Les trois types de décompositions de tâches en planification sont :

1. Décomposition par LLM (Language Large Model) avec des prompts simples tels que "Steps for..." ou "What are the subgoals...".
2. Décomposition basée sur des instructions spécifiques à la tâche, par exemple "Write a story outline" pour l'écriture d'un roman.
3. Décomposition avec des entrées humaines.


### 3. The Rephrasing prompt

There is a 3rd prompt used for the retriever called the *rephrasing_prompt*.
It's used to reformat the user question in a well formatted prompt in order to perform the best similarity search possible


In [21]:
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Initialisation du modèle LLM
llm = init_chat_model("command-r-plus", model_provider="cohere")

# Prompt pour reformuler la question utilisateur
rephrasing_prompt_template = """Your task is to rewrite the user’s question to improve document retrieval.
Make it more precise, explicit, and well-structured while keeping its original meaning.

**Original Question:** {question}

**Rephrased Question:**"""
rephrasing_prompt = PromptTemplate.from_template(rephrasing_prompt_template)

# Prompt pour filtrer les documents
filtering_prompt_template = """Your role is to rate the helpfulness of a document given a user message.
The rating you can give is either True or False.

**Direct definition:** If the document explicitly defines the term or concept asked about in the user message, return 'True'.

**Mention Without Definition:** If the document mentions the term but does not provide a definition or explanation, return 'False'.

**Irrelevant Content:** If the document is unrelated to the user's query, or if the user's message is a greeting or conversational phrase (e.g. 'Hello'), return 'False'.

**When in doubt, Default to False:** If unsure, return 'False' by default.

---

**User Message:** {question}
**Document:** {document}

Respond with only 'True' or 'False'.
"""
filtering_prompt = PromptTemplate.from_template(filtering_prompt_template)

# Prompt principal pour générer la réponse
system_prompt = """You are an AI assistant specialized in Autonomous AI Agents.
You will answer questions based on the retrieved context.

Follow these rules:
1) Respond in the language used by the user.
2) Use only the provided context to formulate your answer.
3) If the context is insufficient, simply state that you do not know.
4) Be clear and concise in your explanations.

Here is the user question and retrieved context:
Question: {question}
Context: {context}
"""
qa_prompt = PromptTemplate.from_template(system_prompt)

# Définition de l'état de l'application
class State(TypedDict):
    question: str
    rephrased_question: str
    context: List[Document]
    answer: str

# Étape 1 : Reformuler la question
def rephrase_question(state: State):
    """Reformuler la question utilisateur pour une meilleure recherche."""
    response = llm.invoke(rephrasing_prompt.format(question=state["question"])).content.strip()
    return {"rephrased_question": response}

# Étape 2 : Récupérer les documents avec la question reformulée
def retrieve(state: State):
    """Récupérer les documents pertinents depuis le vector store."""
    retrieved_docs = vector_store.similarity_search(state["rephrased_question"])  # Utilisation de la question reformulée
    return {"context": retrieved_docs}

# Étape 3 : Filtrer les documents pertinents
def filter_documents(state: State):
    """Filtrer les documents selon leur pertinence en utilisant le filtering prompt."""
    relevant_docs = []
    for doc in state["context"]:
        message = filtering_prompt.format(question=state["question"], document=doc.page_content)
        response = llm.invoke(message).content.strip()

        if response.lower() == "true":
            relevant_docs.append(doc)

    return {"context": relevant_docs}

# Étape 4 : Générer la réponse finale
def generate(state: State):
    """Générer une réponse en utilisant les documents filtrés."""
    if not state["context"]:  # Aucun document pertinent
        return {"answer": "Je ne sais pas, le contexte est insuffisant."}

    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = qa_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compilation du graphe avec rephrasing, récupération, filtrage et génération
graph_builder = StateGraph(State).add_sequence([rephrase_question, retrieve, filter_documents, generate])
graph_builder.add_edge(START, "rephrase_question")
graph = graph_builder.compile()

# Exemple d'utilisation
question = "Quels sont les différents types de décompositions de tâches en planification ?"
response = graph.invoke({"question": question})

# Affichage de la réponse
print(response["answer"])


Les trois types de décompositions de tâches en planification sont :

1. Décomposition par LLM (Language Large Model) avec des prompts simples tels que "Steps for..." ou "What are the subgoals...".
2. Décomposition basée sur des instructions spécifiques à la tâche, par exemple "Write a story outline" pour l'écriture d'un roman.
3. Décomposition avec des entrées humaines.
